In [ ]:
import numpy as np
import pandas as pd
from capping import capping
from coinmarketcap import Market
from bittrex import Bittrex, API_V2_0, API_V1_1
from os import getenv

# get coinmarketcap top 50
coinmarketcap = Market()
data = coinmarketcap.ticker(limit=50)

# create blacklist
blacklist = pd.Series(['BCC','USDT'])
symbol_map = {'IOTA': 'MIOTA',
              'BCC': 'BCH'}

## binance
from binance.client import Client
my_binance = Client(getenv('BINANCE_API_KEY'), getenv('BINANCE_API_SECRET'))
binance_price_btc = pd.Series({t['symbol'].split('BTC')[0]: t['lastPrice'] for t in my_binance.get_ticker() if 'BTC' in t['symbol']})
binance_price_btc.index = binance_price_btc.reset_index()['index'].replace(symbol_map)
binance_price_btc.drop('', inplace=True)
binance_price_btc['BTC'] = 1.0
binance_price_btc.name = 'price_btc'

# binance markets
binance_markets = my_binance.get_products()

binance_coins = pd.Series(list(set([p['baseAsset'] for p in binance_markets['data']])))
binance_coins = binance_coins.replace(symbol_map)

# create dataframe from coinmarketcap data
df = pd.DataFrame(data)

# remove blacklist coins and coins not in bittrex from dataframe
df = df.loc[df['symbol'].isin(binance_coins) & ~df['symbol'].isin(blacklist), :]


# get top 20
df = df.head(20)

# compute market weights
df['market_cap_usd'] = df['market_cap_usd'].astype(float)
df['weight'] = df['market_cap_usd']/df['market_cap_usd'].sum()

# compute capped weights
capped = capping(df, 0.1, weight_column='weight')

weights = capped.loc[:, ['name', 'symbol', 'weight']]#, 'price_btc']]
weights.set_index('symbol', inplace=True)

In [ ]:
my_account = pd.DataFrame([b for b in my_binance.get_account()['balances'] if float(b['free']) > 0])
my_account['asset'] = my_account['asset'].replace(symbol_map)
my_account.set_index('asset', inplace=True)
my_account.drop(['BNB', 'GAS'], inplace=True) # treat this other way

In [ ]:
my_account['current_value'] = my_account['free'].astype(float)
weights = pd.concat((weights, my_account['current_value']), axis=1)
weights = pd.concat((weights, binance_price_btc), axis=1, join='inner')
weights['price_btc'] = weights['price_btc'].astype(float)
weights['current_value_btc'] = weights['current_value'] * weights['price_btc'].astype('float')
weights.fillna(0, inplace=True)
weights['current_weight'] = weights['current_value_btc']/weights['current_value_btc'].sum()
weights['target_value_btc'] = weights['weight'] * weights['current_value_btc'].sum()
weights['diff_btc'] = weights['target_value_btc'] - weights['current_value_btc']
weights['diff_btc_round'] = np.where(weights['diff_btc'].abs() > 0.0002, weights['diff_btc'], weights['diff_btc'].round(3))
weights['diff'] = weights['diff_btc_round']/weights['price_btc']
weights.sort_values('weight', ascending=False)

In [ ]:
def color_negative_red(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    if val < 0:
        color = 'red' 
    elif val > 0:
        color = 'green'
    else:
        color = 'black'
    return 'color: %s' % color

weights.style.applymap(color_negative_red, subset=['diff_btc', 'diff_btc_round', 'diff'])

In [ ]:
diffs = weights.loc[:, ['name', 'price_btc', 'diff_btc_round', 'diff']]

sell_prop = diffs.diff_btc_round.where(diffs.diff_btc_round < 0)/diffs.diff_btc_round.where(diffs.diff_btc_round < 0).sum()
diffs['diff_btc_round'] = np.where(diffs['diff_btc_round'] < 0,  diffs['diff_btc_round']-(diffs['diff_btc_round'].sum()*sell_prop), diffs['diff_btc_round'])
diffs['diff'] = diffs['diff_btc_round']/diffs['price_btc']
#diffs['weight_diff'] = (diffs['diff_btc_round']+weights['current_value_btc'])/(diffs['diff_btc_round']+weights['current_value_btc']).sum()-weights['weight']
print("sum diff_btc_round = ", diffs.diff_btc_round.sum())
diffs.style.applymap(color_negative_red, subset=['diff_btc_round', 'diff'])

In [ ]:
diffs['diff_btc_round'].sum()

In [ ]:
weights.current_value_btc.sum()